In [40]:
pip install  -q openai azure-core azure-search-documents azure-storage-blob tiktoken

StatementMeta(, , -1, Finished, , Finished)

LIVY_JOB_TIMED_OUT: Livy session has failed. Session state: Dead. Error code: LIVY_JOB_TIMED_OUT. Job failed during run time with state=[dead]. Source: Unknown.

In [ ]:
#%run Documentum/logging_utility

In [36]:
import io
import uuid
import json
import math
import tiktoken
from datetime import datetime
from notebookutils import mssparkutils
from openai import AzureOpenAI
from datetime import datetime, timedelta
from azure.search.documents import SearchClient
from azure.storage.blob import BlobServiceClient 
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    SearchIndex,
    SearchField,
    VectorSearch,
    VectorSearchProfile,
    HnswAlgorithmConfiguration,
    SemanticSearch,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField
)
from azure.storage.blob import (
    BlobServiceClient,
    BlobSasPermissions,
    generate_blob_sas,
)

StatementMeta(smallbronzefull, 1877, 41, Finished, Available, Finished)

In [ ]:
service_endpoint = ""
index_name       = "jsonidx"
key              = ""
credential       = AzureKeyCredential(key)
vector_dims       = 1536
algo_name         = "hnsw-cosine"
profile_name      = "openai-ada-profile"



StatementMeta(smallbronzefull, 1877, 42, Finished, Available, Finished)

In [38]:
# Define the index fields
client = SearchIndexClient(service_endpoint, credential)
fields = [
    SimpleField(name="id",   type=SearchFieldDataType.String,key=True, sortable=True, filterable=True, facetable=True),
    SimpleField(name="file_name", type=SearchFieldDataType.String),
    
    SimpleField(name="page_number",type=SearchFieldDataType.Double,sortable=True,filterable=True,facetable=False),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchField(name="content_vector",type=SearchFieldDataType.Collection(SearchFieldDataType.Single),searchable=True,vector_search_dimensions=1536,vector_search_profile_name="my-vector-config"),
    SimpleField(name="storage_url",type=SearchFieldDataType.String,filterable=False,facetable=False,sortable=False)
]
 
vector_search = VectorSearch(
    profiles=[VectorSearchProfile(name="my-vector-config",
                                algorithm_configuration_name="my-algorithms-config")],
    algorithms=[HnswAlgorithmConfiguration(name="my-algorithms-config")],
)

# Define semantic configuration
semantic_config = SemanticConfiguration(
    name="semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="file_name"),
        content_fields=[SemanticField(field_name="content")]
    )
)

# Add semantic search to the index
semantic_search = SemanticSearch(
    default_configuration_name="semantic-config",
    configurations=[semantic_config]
)

# Update the index definition
index = SearchIndex(
    name=index_name,
    fields=fields,
    vector_search=vector_search,
    semantic_search=semantic_search
)

client.create_or_update_index(index)

StatementMeta(smallbronzefull, 1877, 43, Finished, Available, Finished)

In [ ]:
# ─── 1. CONFIG ────────────────────────────────────────────────────────────────
AZURE_OPENAI_ENDPOINT  = ""
AZURE_OPENAI_KEY       = ""
AZURE_OPENAI_API_VER   = "2024-10-21"
EMBED_DEPLOYMENT       = "emb"
 
SEARCH_ENDPOINT        = ""
SEARCH_API_KEY         = ""
SEARCH_INDEX_NAME      = "jsonidx"
 
ADLS_CONNECTION_STRING = ""  # blob endpoint
ADLS_CONTAINER_NAME    = "bronze"
ADLS_DIR_PREFIX        = "pdf/"
ADLS_STORAGE_ACCOUNT  = ""
ADLS_STORAGE_KEY = ""

 
TEXT_MIN_CHARS         = 5
 
# ─── 2. CLIENTS ───────────────────────────────────────────────────────────────
openai_client = AzureOpenAI(
    api_key        = AZURE_OPENAI_KEY,
    azure_endpoint = AZURE_OPENAI_ENDPOINT,
    api_version    = AZURE_OPENAI_API_VER,
)
 
search_client = SearchClient(
    endpoint   = SEARCH_ENDPOINT,
    index_name = SEARCH_INDEX_NAME,
    credential = AzureKeyCredential(SEARCH_API_KEY)
)
 
blob_service  = BlobServiceClient.from_connection_string(ADLS_CONNECTION_STRING)
 
container     = blob_service.get_container_client(ADLS_CONTAINER_NAME)
 
def create_embedding(text: str, model: str = EMBED_DEPLOYMENT) -> list[float]:

    return openai_client.embeddings.create(input=[text], model=model).data[0].embedding
 

tokenizer = tiktoken.encoding_for_model("text-embedding-ada-002")

MAX_TOKENS = 8192
 
def page_records_from_bytes(json_bytes: bytes):
    """
    Yield (page_no, text) tuples where `text` contains:
      - the joined `lines[]`
      - every table rendered as "col1: val1 | col2: val2" per row
      - split into sub-pages if over MAX_TOKENS
    """
    data = json.loads(json_bytes.decode("utf-8"))
    if not isinstance(data, dict) or "pages" not in data:
        return  # nothing to emit
    for page in data["pages"]:
        page_no = page.get("page_number", 0)
        # 1. Extract lines
        lines = page.get("lines", [])
        text_blocks = ["\n".join(str(l).strip() for l in lines if str(l).strip())]
        # 2. Extract tables
        for tbl in page.get("tables", []):
            for row in tbl.get("rows", []):
                if isinstance(row, dict):
                    row_txt = " | ".join(f"{k}: {v}" for k, v in row.items())
                    text_blocks.append(row_txt.strip())
        # 3. Final full text block
        full_text = "\n".join(t for t in text_blocks if t).strip()
        if not full_text:
            continue
        # 4. Token-aware chunking
        tokens = tokenizer.encode(full_text)
        total_tokens = len(tokens)
        if total_tokens <= MAX_TOKENS:
            yield str(page_no), full_text
        else:
            num_chunks = math.ceil(total_tokens / MAX_TOKENS)
            for i in range(num_chunks):
                start = i * MAX_TOKENS
                end = start + MAX_TOKENS
                chunk_tokens = tokens[start:end]
                chunk_text = tokenizer.decode(chunk_tokens)
                chunked_page_no = f"{page_no}.{i+1}"
                yield chunked_page_no, chunk_text
 
def make_sas_url(file_name: str) -> str:
    sas = generate_blob_sas(
        account_name=ADLS_STORAGE_ACCOUNT,
        container_name=ADLS_CONTAINER_NAME,
        blob_name=file_name,
        account_key=ADLS_STORAGE_KEY,
        permission=BlobSasPermissions(read=True),
        expiry=datetime.utcnow() + timedelta(days=7)
    )
    return f"https://{ADLS_STORAGE_ACCOUNT}.blob.core.usgovcloudapi.net/{ADLS_CONTAINER_NAME}/{file_name}?{sas}"
 
# ─── MAIN (only the loop body changes) ────────────────────────────────────────

def main():
    uploaded = skipped = 0
    total_json_parsed = 0
    total_json = 0
    page_parsed = 0
    json_size =0
    print("➜ Scanning ADLS container for JSON files …")
    start = datetime.utcnow()
    for blob in container.list_blobs():
        if not blob.name.lower().endswith(".json"):
            continue
 
        print(f"\n➜ Processing {blob.name}")

        json_bytes = container.download_blob(blob.name).readall()
        total_json += 1
        json_size += len(json_bytes)
        for page_no, text in page_records_from_bytes(json_bytes):

            if len(text) < TEXT_MIN_CHARS:
                skipped += 1
                continue
 
            doc = {
                "id":             str(uuid.uuid4()),
                "file_name":      blob.name,
                "page_number":    page_no,
                "content":        text,
                "content_vector": create_embedding(text),
                "storage_url" : make_sas_url(blob.name)
            }

            result  = search_client.upload_documents([doc])[0]

            status  = "✓" if result.succeeded else "✗"
            if(status == "✓"):
                page_parsed += 1
            uploaded += int(result.succeeded)
        if(page_parsed > 0):
            total_json_parsed +=1
        print(f"Total page parsed {page_parsed}")
        page_parsed = 0
 
    print(f"\nDone. {uploaded} pages indexed, {skipped} pages skipped.")
    print(f"Total JSON processed: {total_json_parsed}\n Total JSON in Raw: {total_json}" )
    #end = datetime.utcnow()
    #pid = str(uuid.uuid4())
    #stage = "Bronze to Silver"
    #status = "Convert JSON to AI"
    #size = json_size
    #log_activity(pid, stage, status,start,end, size)
 
if __name__ == "__main__":

    main()
 

StatementMeta(smallbronzefull, 1877, 44, Finished, Available, Finished)

➜ Scanning ADLS container for JSON files …

➜ Processing  12B O-6.9.2, CDP, Rev-038, PCR-24-01396, GLH.pdf.json
Total page parsed 66

➜ Processing  Lessons Learned - Information Bulletin.json
Total page parsed 1

➜ Processing  REMOVE SIGHT GLASSES FROM SERVICE P-15.3 ESP Rev-003.pdf.json
Total page parsed 10

➜ Processing 003.R0, Rev 000, RBS CYCLE 10 A2-2 SEQUENCE ESTIMATED CRITICAL POSITIONS AND NOTCH WORTH CALCULATIONS.json


HttpResponseError: () The request is invalid. Details: Invalid JSON. A token was not recognized in the JSON content.
Code: 
Message: The request is invalid. Details: Invalid JSON. A token was not recognized in the JSON content.